In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('lesser_model_data.csv')


In [3]:
def get_lesser_model_states():
    statedataStayHome = {'MarHome' : data['MarHome'], 'AprHome' : data['AprHome'], 'MayHome' : data['MayHome'], 'JunHome' : data['JunHome']}
    statesStayHome = pd.DataFrame(data=statedataStayHome)

    statedataTests = {'MarTest' : data['MarTest'], 'AprTest' : data['AprTest'], 'MayTest' : data['MayTest'], 'JunTest' : data['JunTest']}
    statesTests = pd.DataFrame(data=statedataTests)

    statedataCases = {'MarCases' : data['MarCases'], 'AprCases' : data['AprCases'], 'MayCases' : data['MayCases'], 'JunCases' : data['JunCases']}
    statesCases = pd.DataFrame(data=statedataCases) # 0 = increasing. 1 = flat or decreasing
    
    return statesStayHome, statesTests, statesCases


In [5]:
def get_probabilities(states):
    num_total = np.shape(states)[0] * np.shape(states)[1]
    
    num_ones = np.shape(np.where(states == 1))[1]
    num_zeros = num_total - num_ones

    prob_one = num_ones/num_total
    prob_zero = num_zeros/num_total
    
    assert round(prob_one+prob_zero, 3) == 1. 
    
    return prob_one, prob_zero

In [12]:
def get_conditional_probability(Astates, Bstates, Cstates):
    """A and B are parent nodes. C is child node"""
    
    num111, num101, num011, num001, num110, num100, num010, num000 = 0, 0, 0, 0, 0, 0, 0, 0
    num11, num01, num10, num00 = 0,0,0,0
    
    
    #Add up cases where C=1
    indC_one_row = np.where(Cstates == 1)[0] #index of row with a 1
    indC_one_col = np.where(Cstates == 1)[1] #index of corresponding coln with a 1

    for i in range(len(indC_one_col)): # loop through all times C=1
        A = Astates.iloc[indC_one_row[i], indC_one_col[i]]
        B = Bstates.iloc[indC_one_row[i], indC_one_col[i]]
        if A == 1:
            if B == 1:
                num111 += 1  #C=1, B=1, A=1
                num11 += 1  #B=1, A=1
            elif B == 0:
                num101 += 1  #C=1, B=0, A=1
                num01 += 1  #B=0, A=1
        elif A == 0:
            if B == 1:
                num110 += 1  #C=1, B=1, A=0
                num10 += 1  #B=1, A=0
            elif B == 0:
                num100 += 1  #C=1, B=0, A=0
                num00 += 1  #B=0, A=0
                
    # Add up states where C=0
    indC_zero_row = np.where(Cstates == 0)[0] 
    indC_zero_col = np.where(Cstates == 0)[1] 


    for i in range(len(indC_zero_col)): # loop through all times C=0
        A = Astates.iloc[indC_zero_row[i], indC_zero_col[i]]
        B = Bstates.iloc[indC_zero_row[i], indC_zero_col[i]]
        if A == 1:
            if B == 1:
                num011 += 1  #C=0, B=1, A=1
                num11 += 1  #B=1, A=1
            elif B == 0:
                num001 += 1  #C=0, B=0, A=1
                num01 += 1  #B=0, A=1
        elif A == 0:
            if B == 1:
                num010 += 1  #C=0, B=1, A=0
                num10 += 1  #B=1, A=0
            elif B == 0:
                num000 += 1  #C=0, B=0, A=0
                num00 += 1  #B=0, A=0
    
    # Calculate conditional probabilities
    num_total = np.shape(Astates)[0] * np.shape(Astates)[1]
    
    """The variable names are in the order CBA (cases, testing, stay at home). """
    P111 = num111/num11
    P101 = num101/num01
    P110 = num110/num10
    P100 = num100/num00
    P011 = num011/num11
    P001 = num001/num01
    P010 = num010/num10
    P000 = num000/num00
    
    states = np.array(['0|00','1|00','0|01','1|01','0|10','1|10','0|11','1|11']) #I believe this is the order Matt was talking about with top to bottom, left to right in the table from the paper
    conditional_probabilities = np.array([P000,P100,P001,P101,P010,P110,P011,P111]) #in this order, the first two, second two, etc. should each add to 1 as pairs (columns in the paper's table)
    
    # Make graph for circuit input
    ProbA1, ProbA0 = get_probabilities(Astates)
    ProbB1, ProbB0 = get_probabilities(Bstates)
    
    graph = { 'StayAtHome': ([], [ProbA0, ProbA1]), # P(A = 0), P(A = 1)
                'Testing': ([], [ProbB0, ProbB1]),  #P(B = 0), P(B = 1)
                #P(C=0|A=0,B=0), P(C=1|A=0,B=0), P(C=0|A=0,B=1), P(C=1|A=0,B=1), P(C=0|A=1,B=0), P(C=1|A=1,B=0), P(C=0|A=1,B=1), P(C=1|A=1,B=1)
                'Cases': (['StayAtHome','Testing'], [P000, P100, P010, P110, P001, P101, P011, P111])
              }
    
    return states, conditional_probabilities, graph